In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random

plt.rcParams["axes.grid"] = False
plt.style.use('dark_background')

In [3]:
df = pd.read_csv('train2.csv')
df.head()

,filename,opacity,diabetic retinopathy,glaucoma,macular edema,macular degeneration,retinal vascular occlusion,normal,new
0,c24a1b14d253.jpg,0,0,0,0,0,1,0,retinal vascular occlusion
1,9ee905a41651.jpg,0,0,0,0,0,1,0,retinal vascular occlusion
2,3f58d128caf6.jpg,0,0,1,0,0,0,0,glaucoma
3,4ce6599e7b20.jpg,1,0,0,0,1,0,0,"opacity, macular degeneration"
4,0def470360e4.jpg,1,0,0,0,1,0,0,"opacity, macular degeneration"


In [20]:
df.shape

(3147, 9)

In [3]:
df.new.value_counts()

opacity                                             671
normal                                              525
glaucoma                                            327
macular degeneration                                299
opacity, macular degeneration                       212
diabetic retinopathy                                212
opacity, diabetic retinopathy                       186
retinal vascular occlusion                          170
diabetic retinopathy, macular edema                 162
opacity, glaucoma                                   140
macular edema, retinal vascular occlusion            74
opacity, retinal vascular occlusion                  55
glaucoma, macular degeneration                       31
glaucoma, retinal vascular occlusion                 27
macular edema                                        27
diabetic retinopathy, glaucoma                       15
diabetic retinopathy, retinal vascular occlusion      5
glaucoma, macular edema                         

### Experiment 1
- Upsample/Downsample only the single cases to 500 and see, if double cases will still be recognized

### Experiment 2
- Labels with 2, 3 and 5 Instances: delete 14 rows
- Labels with 15-74 Instances: upsampling to 400 with rotation+/-, horzontal flip, vertical flip, channel shift (15x26=400) 
- Labels with more than 140 instances: upsampling to 400
- Labels with more than 500 instances: downsampling to 400

## Experiment 1

### Downsample normal and opacity

In [5]:
# randomly delete images with only opacity to downsample to 500 and drop corresponding row
#Fehler: np.random hat eine Datei 23 mal ausgesucht. replace=False sollte helfen, hat es aber nicht
#rand_im = [np.random.choice(images, replace=False) for i in range(171)]
#rand_im.index('86a2f4eb36ce.jpg')

In [ ]:
# randomly delete images with only normal to downsample to 500 and drop corresponding row

In [18]:
#downsample('normal', 25)

['c50c4eda2979.jpg', 'e0ec0bd72219.jpg', '91a41dd87d24.jpg', '982a69e7e643.jpg', '8df428037090.jpg', '26517e8ee001.jpg', '80281991a0d1.jpg', '57a895b2e5e8.jpg', '3a58fc3e2f68.jpg', 'c022888ee894.jpg', '48a2b5ae3ed9.jpg', '97ea1365c513.jpg', '4404cccb8434.jpg', 'ed1294df0216.jpg', '256f1cd27c22.jpg', 'e23fafd467ad.jpg', '3330a27f96fd.jpg', 'f68d64049e4a.jpg', '44848f205e96.jpg', 'a56e4d805912.jpg', 'f67f10975eef.jpg', '21fa12b5d219.jpg', '9c9ea0f8f7bf.jpg', '738684d432b7.jpg', '9261f02bedb0.jpg']


In [19]:
#downsample('opacity', 171)

['09a58f5d52c4.jpg', '7c627414b595.jpg', '41a4c94eb846.jpg', 'e87d2eb72f62.jpg', 'bc8ce4660640.jpg', 'fab49a013c13.jpg', 'ae1656d96d18.jpg', 'eef26fbd8061.jpg', '80009967474e.jpg', '33dfbf13fd75.jpg', '244b85b1de3f.jpg', '2a830198259b.jpg', '7b39fb001764.jpg', '7ef1ac9fddc2.jpg', 'ccd7e6bfef58.jpg', 'c138cca7bd0c.jpg', '93014fc04656.jpg', '59612b54b41b.jpg', '3b8080fb2d34.jpg', '764b008ec26f.jpg', '6fbb95a10d8e.jpg', 'ed205e61f072.jpg', 'd63c87e5c7a3.jpg', 'dbaae430ffc3.jpg', '0056b75f7627.jpg', '2f41bed25d94.jpg', '40b0da976d9b.jpg', 'ba1a5cb5b139.jpg', 'fe82dd356325.jpg', '3d1a96bced4f.jpg', '766dfd258afd.jpg', '4ad9d8a014f2.jpg', 'b13429b39ff0.jpg', 'ccbe09dcf066.jpg', '1dff472aff29.jpg', '909925f2bbee.jpg', 'ae008545ce81.jpg', '3e519424900e.jpg', 'b8b65f9121e4.jpg', '5f581f1dee99.jpg', '26493786f81b.jpg', '6683cd1c9b3b.jpg', '5e374e4bd008.jpg', '7ee2fa2f4019.jpg', 'ede3a0953ac6.jpg', '677a44962504.jpg', '15bbd165c88b.jpg', 'af2b0b5c519a.jpg', '3f0c768b764e.jpg', 'bb95b34af7a9.jpg',

In [11]:
uber_lange_liste = ['09a58f5d52c4.jpg', '7c627414b595.jpg', '41a4c94eb846.jpg', 'e87d2eb72f62.jpg', 'bc8ce4660640.jpg', 'fab49a013c13.jpg', 'ae1656d96d18.jpg', 'eef26fbd8061.jpg', '80009967474e.jpg', '33dfbf13fd75.jpg', '244b85b1de3f.jpg', '2a830198259b.jpg', '7b39fb001764.jpg', '7ef1ac9fddc2.jpg', 'ccd7e6bfef58.jpg', 'c138cca7bd0c.jpg', '93014fc04656.jpg', '59612b54b41b.jpg', '3b8080fb2d34.jpg', '764b008ec26f.jpg', '6fbb95a10d8e.jpg', 'ed205e61f072.jpg', 'd63c87e5c7a3.jpg', 'dbaae430ffc3.jpg', '0056b75f7627.jpg', '2f41bed25d94.jpg', '40b0da976d9b.jpg', 'ba1a5cb5b139.jpg', 'fe82dd356325.jpg', '3d1a96bced4f.jpg', '766dfd258afd.jpg', '4ad9d8a014f2.jpg', 'b13429b39ff0.jpg', 'ccbe09dcf066.jpg', '1dff472aff29.jpg', '909925f2bbee.jpg', 'ae008545ce81.jpg', '3e519424900e.jpg', 'b8b65f9121e4.jpg', '5f581f1dee99.jpg', '26493786f81b.jpg', '6683cd1c9b3b.jpg', '5e374e4bd008.jpg', '7ee2fa2f4019.jpg', 'ede3a0953ac6.jpg', '677a44962504.jpg', '15bbd165c88b.jpg', 'af2b0b5c519a.jpg', '3f0c768b764e.jpg', 'bb95b34af7a9.jpg', '79c63b3ed8ae.jpg', 'e82f99781ae8.jpg', '8451465343df.jpg', 'b415a1e39b15.jpg', '3f78c1d49341.jpg', 'fc8defa9a696.jpg', '592c8f680c46.jpg', 'f040a336e19c.jpg', 'e733b2b38188.jpg', 'ffea3b0abccf.jpg', 'b003d44c48d3.jpg', '8294533ce2c3.jpg', 'f66ead1505c2.jpg', '7e43317bd835.jpg', '76912935cfd8.jpg', '98c149511cda.jpg', '28fbf39f96f8.jpg', '012e91949ec4.jpg', 'dfa0d81dca91.jpg', '50c7fdd1e6ed.jpg', '9510d0867d9a.jpg', 'fe39996c8267.jpg', '3cf275051cb8.jpg', '16ef2d42c24b.jpg', 'f9378c7ebf0b.jpg', '06a6c5d1a41e.jpg', 'e6d066d41b9b.jpg', '9abd0ea81b3b.jpg', '17b7942e6561.jpg', 'e43722f4fef0.jpg', '53d8aef79977.jpg', '6a89e15fcce3.jpg', 'c878e6a45e1b.jpg', 'febce91521dc.jpg', 'de952deada91.jpg', '5156411c155a.jpg', '53719f727403.jpg', 'cbda59260880.jpg', 'c7d15985bfd0.jpg', 'efb1ae7db959.jpg', 'fafe8d6db2b3.jpg', '02840e6a14e2.jpg', '1d22c859b7fd.jpg', '2067e7554ee2.jpg', '6431d94b6c54.jpg', '3907f7070535.jpg', 'f3ec24dadd3e.jpg', '03ce7c893a99.jpg', 'd2827987e7e6.jpg', '436557ff1340.jpg', '1d1b8aa41b39.jpg', 'f7eced961a9b.jpg', '29dcdcd5eea4.jpg', 'c8b9637cd338.jpg', '06c3122899e7.jpg', '6d005a60745f.jpg', 'c3bce73b4a49.jpg', 'f0055237f646.jpg', 'e27edbbc8456.jpg', 'b5b1e4c722b4.jpg', '09d81f09c306.jpg', 'a8b33fbd89ce.jpg', '86a2f4eb36ce.jpg', 'd5f8eceb6263.jpg', 'c0282aa6609f.jpg', '8943c0133278.jpg', 'ac713827c7cb.jpg', '50750b6d117c.jpg', 'ff56c87808fa.jpg', 'e901d2dece0f.jpg', 'e9189b155c9b.jpg', 'bebbb26d1f1d.jpg', '306afd7b9ebf.jpg', '7728bf2577f3.jpg', 'fb219babc648.jpg', '2f2fa76788f3.jpg', 'd631c81e51f2.jpg', '3cc790793d9f.jpg', '78406135ccdb.jpg', '582e9f7af2ae.jpg', '1a2567b7cb1b.jpg', 'b70eb39c9e4a.jpg', '88ac46f5f21b.jpg', '22c0da6369c0.jpg', 'c922c538915e.jpg', '89b2f8a5df5b.jpg', 'e8cd8fe9a858.jpg', '73cb0e282cb0.jpg', '89f52c955039.jpg', '4753439c7f46.jpg', 'd2adc26acebc.jpg', '3c2c970c466d.jpg', 'd7166dc96f4b.jpg', '18b78cf415c1.jpg', '6fbfbe164a0e.jpg', '7ed3f9e6e52f.jpg', 'a85226fe5d5b.jpg', 'a764e26bcf8e.jpg', '86a591a2fe43.jpg', 'e1fa01dcaef3.jpg', 'a871a1fe5222.jpg', 'b348efa764af.jpg', '7f47121ab858.jpg', '6427c74113ad.jpg', '9049590eef6b.jpg', 'ae1f1fb9af6e.jpg', '8a7ac84558f8.jpg', 'eec52dffcecc.jpg', 'f8e3ad5ec600.jpg', '2760ad30a5c0.jpg', 'eee73cb8aeb1.jpg', '2401c17651ba.jpg', '9ed97c08cc66.jpg', '1335a7fa63ef.jpg', '1868f896740e.jpg', 'ba2f4900c169.jpg', '94973b732f82.jpg', 'ad650b2c1bfe.jpg', 'c989dc002c81.jpg', 'a7b493d32dfd.jpg', '875ee3025749.jpg']

In [4]:
lange_liste = ['c50c4eda2979.jpg', 'e0ec0bd72219.jpg', '91a41dd87d24.jpg', '982a69e7e643.jpg', '8df428037090.jpg', 
               '26517e8ee001.jpg', '80281991a0d1.jpg', '57a895b2e5e8.jpg', '3a58fc3e2f68.jpg', 'c022888ee894.jpg', 
               '48a2b5ae3ed9.jpg', '97ea1365c513.jpg', '4404cccb8434.jpg', 'ed1294df0216.jpg', '256f1cd27c22.jpg', 
               'e23fafd467ad.jpg', '3330a27f96fd.jpg', 'f68d64049e4a.jpg', '44848f205e96.jpg', 'a56e4d805912.jpg', 
               'f67f10975eef.jpg', '21fa12b5d219.jpg', '9c9ea0f8f7bf.jpg', '738684d432b7.jpg', '9261f02bedb0.jpg']

In [24]:
# bild in ordner suchen, wenn nicht da, in dataframe löschen
# wenn funktion bild löscht, gleich reihe in dataframe löschen

In [15]:
# entweder liste von oben nutzen, um dataframe zu reduzieren
df_new = df[~df.filename.isin(lange_liste)]
df_new = df_new[~df.filename.isin(uber_lange_liste)]

<ipython-input-15-96733c3f96de>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_new = df_new[~df.filename.isin(uber_lange_liste)]


In [16]:
df_new.shape

(2951, 9)

In [17]:
df_new.new.value_counts()

normal                                              500
opacity                                             500
glaucoma                                            327
macular degeneration                                299
diabetic retinopathy                                212
opacity, macular degeneration                       212
opacity, diabetic retinopathy                       186
retinal vascular occlusion                          170
diabetic retinopathy, macular edema                 162
opacity, glaucoma                                   140
macular edema, retinal vascular occlusion            74
opacity, retinal vascular occlusion                  55
glaucoma, macular degeneration                       31
macular edema                                        27
glaucoma, retinal vascular occlusion                 27
diabetic retinopathy, glaucoma                       15
diabetic retinopathy, retinal vascular occlusion      5
glaucoma, macular edema                         

In [18]:
# downgesampleten datensatz speichern
df_new.to_csv('train3.csv', sep=',', encoding='utf-8', index=False)